# ***Finding Case Outcomes and Labelling Data***

In this notebook, i aim to try and isolate case outcomes from the documents.csv
file manually, as well as using the RECAP open sources PACER data.

For missing outcomes that I fail to match, it might be useful idea to train LegalBERT on a downstream classification task using the labelled data and then using predicted case outcomes in my regressions.

In [1]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#loading up google drive to access csv files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
#loading my csv file
documents = pd.read_csv('/content/drive/MyDrive/ECO225_Final_Project/PART1/documents.csv')

<ipython-input-8-76cc3cccd0bc>:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  documents = pd.read_csv('/content/drive/MyDrive/ECO225_Final_Project/PART1/documents.csv')


In [17]:
#isolating relavent variables
clean_docs = documents.copy()
clean_docs = clean_docs.drop(columns=['case_number_raw', 'attachment', 'doc_date_filed', 'doc_date_uploaded', 'document_url'])

#creating column that replaces NaN long_description values with short_description's
clean_docs['combined_description'] = clean_docs['long_description'].fillna(clean_docs['short_description'])


#dropping all missiing long_descriptions
clean_docs = clean_docs.dropna(subset=['combined_description'])

In [25]:
# Set the display option to show full column contents
pd.set_option('display.max_colwidth', None)

clean_docs.head()

,case_row_id,case_number,district_id,doc_count,doc_number,short_description,long_description,combined_description
0,1.0,0:79-cv-06704,flsd,1,37.0,NaN,"COPY OF PAPER DOCKET SHEET (kw, Deputy Clerk) (Entered: 08/03/2000)","COPY OF PAPER DOCKET SHEET (kw, Deputy Clerk) (Entered: 08/03/2000)"
1,1.0,0:79-cv-06704,flsd,2,NaN,NaN,"CASE CLOSED. Case and Motions no longer referred to Magistrate. (kw, Deputy Clerk) (Entered: 08/03/2000)","CASE CLOSED. Case and Motions no longer referred to Magistrate. (kw, Deputy Clerk) (Entered: 08/03/2000)"
2,3.0,0:83-cv-06860,flsd,1,123.0,NaN,COPY OF PAPER DOCKET SHEET (Former Deputy Clerk) (Entered: 02/13/2004),COPY OF PAPER DOCKET SHEET (Former Deputy Clerk) (Entered: 02/13/2004)
3,3.0,0:83-cv-06860,flsd,2,NaN,NaN,Case closed (Former Deputy Clerk) (Entered: 03/05/1992),Case closed (Former Deputy Clerk) (Entered: 03/05/1992)
4,3.0,0:83-cv-06860,flsd,3,NaN,NaN,Case reopened (Former Deputy Clerk) (Entered: 05/13/1991),Case reopened (Former Deputy Clerk) (Entered: 05/13/1991)


---
# STEP 1: Manually Classify Based on long_descriptions that Include Judgements

This will serve as a high accuracy base line to train/test NLP models used later down the road:


In [43]:
# Define judgment-related keywords
judgment_keywords = ["judgment", "judgement"]

# Filter rows where any keyword appears in the combined_description column
judgment_mask = clean_docs['combined_description'].str.contains(
                      '|'.join(judgment_keywords), case=False, na=False)

# Resetting index to access columns by name
judgment_df = clean_docs[judgment_mask][['case_row_id', 'case_number',
                                         'doc_count', 'long_description',
                                         'combined_description']]


#first word should be "FINAL" - additional accuracy filter
clean_judgments = pd.DataFrame()
clean_judgments = judgment_df[judgment_df['long_description'
                                    ].str.split().str[0] == "FINAL"]

In [44]:
clean_judgments = clean_judgments.set_index(['case_row_id', 'case_number', 'doc_count']).sort_index()
clean_judgments.head(10)

long_description  \
case_row_id case_number   doc_count                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
3.0         0:83-cv-06860 5                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           FINAL JUDGMENT for Kenneth R. Cornwall against U. S. Construction. Defendant permanently enjoined from manufacturing, advertising, etc. any device infringing patent; Permanent injunction effective as of date of entry of judgment; plaintiff shall recover from principal sum of $13,647.42 with pre-judgment interest from 4/30/85 thru 7/7/87 in the amount of $95.00 FOR WHICH LET EXECUTION ISSUE. ( signed by Judge Jose A. Gonzalez Jr. on 7/21/87) CCAP/M (Former Deputy Clerk) (Entered: 05/13/1991)   
53.0        0:91-cv-06658 13                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       FINAL JUDGMENT entered for Clotilde, Inc. and against Solar-Kist Corporation on the complaint and judgment is entered for defendant-counter-plaintiff on the counterclaim. Patents (4,320,699 and 4,597,812) are declared invalid. ( signed by Judge Kenneth L. Ryskamp on 9/8/93) CCAP/M (ea, Deputy Clerk) (Entered: 09/14/1993)   
72.0        0:93-cv-06063 3                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               FINAL JUDGMENT Dismissi